In [1]:
from ophelia_spark.start import Ophelia

In [2]:
ophelia = Ophelia("Ophelia-Test", no_mask=True)

02:22:06.524 Ophelia [INFO] Ophelia Active Session
02:22:06.524 Ophelia [INFO] Ophelia Active Session
02:22:06.524 Ophelia [INFO] Ophelia Active Session
02:22:06.887 Ophelia [TAPE] +-----------------------------------------------------------------+
02:22:06.888 Ophelia [INFO] | Hello! This engine is for data mining & ml pipelines in PySpark |
02:22:06.888 Ophelia [INFO] | Welcome to Ophelia Spark miner engine                           |
02:22:06.888 Ophelia [INFO] | Package Version Ophelia.0.0.1                                   |
02:22:06.888 Ophelia [WARN] | V for Vendata...                                                |
02:22:06.888 Ophelia [TAPE] +-----------------------------------------------------------------+
02:22:06.888 Ophelia [WARN]                       - Ophelia Spark -                        
02:22:06.888 Ophelia [MASK]   █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █ █
02:22:06.888 Ophelia [MASK]   █ █ █ █ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ ╬ █ █

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/18 02:22:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/07/18 02:22:08 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


02:22:08.639 Ophelia [INFO] Spark Version: 3.2.2
02:22:08.639 Ophelia [INFO] This Is: 'Ophelia-Test' App
02:22:08.640 Ophelia [INFO] Spark UI Address: http://192.168.1.70:4041
02:22:08.640 Ophelia [INFO] Spark Context Initialized Successfully


In [3]:
import pandas as pd

print(pd.__version__)

1.3.5


In [4]:
sc = ophelia.Spark.build_spark_context()
spark = ophelia.SparkSession

02:22:54.030 Ophelia [INFO] Spark Context Initialized Successfully


In [5]:
from ophelia_spark.read.spark_read import Read

In [6]:
sample_bank_df = spark.readFile("data/csv/100sample_bank", "csv")
sample_bank_df.show()

02:22:56.564 Ophelia [INFO] Read CSV File From Path: data/csv/100sample_bank
+---+-------------+-------+---------+-------+-------+--------+--------+
|age|          job|marital|education|housing|balance|duration|poutcome|
+---+-------------+-------+---------+-------+-------+--------+--------+
| 36| entrepreneur|married|secondary|     no|   2971|     247| failure|
| 30|     services| single|  primary|    yes|   5746|     125| unknown|
| 29|  blue-collar| single|secondary|     no|   2806|     294|   other|
| 35|   technician|married|secondary|     no|      0|     125| unknown|
| 30|   management| single| tertiary|     no|    697|     179| success|
| 33|       admin.|married|secondary|     no|      0|     214| unknown|
| 31|   management| single| tertiary|     no|   1308|     323| unknown|
| 55|    housemaid|married|  primary|     no|      4|      85| unknown|
| 59|      retired|married|secondary|     no|    136|     301| unknown|
| 34|self-employed| single|secondary|    yes|      3|     3

In [ ]:
from ophelia_spark.functions import DynamicSampling

In [ ]:
sample_df = sample_bank_df.simple_sample(10)
sample_df.show()

In [ ]:
from schemas.fake_schemas import bank_schema

In [ ]:
sample_df.coalesce(1).write.mode("overwrite").csv(
    "data/csv/100sample_bank", header=True, sep=","
)

In [ ]:
spark.read.csv("data/csv/100sample_bank", schema=bank_schema, header=True).show()

In [ ]:
sample_schema = sample_df.schema
spark.read.csv("data/csv/100sample_bank", sample_schema).show()

In [ ]:
import pandas as pd
from data.fake_datasets import sales_dataset

In [ ]:
sales_dataset

In [ ]:
spark.createDataFrame(sales_dataset)

In [ ]:
df = spark.createDataFrame(pd.DataFrame(data=sales_dataset))
df.show(5)

In [ ]:
from ophelia.functions import Shape

df.Shape

In [ ]:
from ophelia.functions import PctChange

df.pctChange().show(5, False)

In [ ]:
df.pctChange(periods=2).na.fill(0).show(5, False)

In [ ]:
df.pctChange(partition_by="Product", order_by="Year", pct_cols="Revenue").na.fill(
    0
).show(5, False)

In [ ]:
df.pctChange(
    partition_by="Product", order_by="Year", pct_cols=["Year", "Revenue"]
).na.fill(0).show(5, False)

In [ ]:
from ophelia.functions import Reshape

# We can choose either one column to aggregate with one single operation
agg_dict_test = {"Revenue": "max"}

# Or moreTthan one column and more than one operation
_agg_dict_test = {
    "Revenue": "mean, count, sum, stddev, var, min",
    "Year": "count, sum",
    "Product": "count",
}

df.toWide(group_by="Product", pivot_col="Year", agg_dict=agg_dict_test).orderBy(
    "Product"
).show(5, False)

In [ ]:
from data.fake_datasets import bank_dataset

In [ ]:
bank_dataset

In [ ]:
bank_df_pd = pd.DataFrame(bank_dataset)

In [ ]:
print(pd.__version__)

In [ ]:
!pip install pandas==1.3.5

In [ ]:
print(pd.__version__)

In [ ]:
bank_df = spark.createDataFrame(bank_df_pd)

In [ ]:
bank_pdf = pd.DataFrame(bank_dataset)

bank_df = spark.createDataFrame(bank_pdf)
bank_df.show()

In [ ]:
from ophelia_spark.functions import Selects
from ophelia_spark.generic import regex_expr

bank_df.selectRegex(regex_expr(["day", "ous", "aig"])).show(5, False)

In [ ]:
bank_df.selectContains(["day", "ous", "aig"]).show(5, False)

In [ ]:
from ophelia.generic import string_match

bank_df.where(string_match("housing == yes")).crosstab("education", "campaign").orderBy(
    "education_campaign"
).show()

In [ ]:
test_agg_dict_ = {"balance": "sum"}
bank_df.where(string_match("housing == yes")).toWide(
    "education", "campaign", test_agg_dict_
).orderBy("education_campaign").show()

In [ ]:
from ophelia.functions import MapItem

map_values = {"A": "0", "B": "1", "C": "2"}

df.mapItem("Product", "bin_product", map_values).show()